# Implementación del algoritmo Firefly

En esta libreta se presenta la implementacion del algoritmo de Firefly para optimización.

El primer paso es importar las librerías necesarias para manipular datos, números aleatorios, graficar, etc.

In [1]:
# Manejo de arreglos y operaciones matematicas
import numpy as np

# Importar funciones
import import_ipynb
from utils import FuncionesObjetivo as f

# Numeros aleatorios
import random

# Para visualizar las animaciones
%matplotlib notebook

importing Jupyter notebook from /Users/isaacg/Desktop/Maestría/Primer_Semestre/Algoritmos_Bioinspirados/Firefly/utils/FuncionesObjetivo.ipynb


## Definir Hiperparámetros del algoritmo

Ahora se asignan los valores para los hiperparámetros del algoritmo como la atracción inicial, el coeficiente de absorcion, y la mutacion

In [4]:
# ------------------ Del Problema ------------------
d = 4  # Numero de dimensiones
funcion = f.f_colville # funcion objetivo
rango = [[-10, 10]]  # Rango de las variables
mark = [0, 0, 0]  # Minimo real (si se conoce)

# ------------------ Del Algoritmo ------------------
n = 50  # Numero de particulas
b0 = 1  # atraccion inicial
gamma = 0.01  # Coeficiente de absorcion
alpha = 1  # mutacion
max_gen = 1000  # Maximo de generaciones
tol = 1e-2  # Tolerancia para detener el algoritmo

# ------------------ Del visualizacion ------------------
animacion = True  # Realizar animacion o no
levels = 20 # numero de curvas de nivel

## Implementacion del algoritmo

Ahora se implementa el algoritmo de firefly usando las funciones y los hiperparámetros definidos anteriormente

In [5]:
# Si solo hay un rango se considera el mismo rango para todas las variables
if len(rango) == 1:
    rango *= d
# Convertir rango a np.array
rango = np.array(rango)

gen = 0

# Crear un vetor de soluciones para cada dimension con su rango dado
X = np.random.uniform(low=rango[:, 0], high=rango[:,1], size=((n, d)))

# Inicializar la intensidad de las particulas con el fitness
I = funcion(X)

# Definimos un mejor fitness para iniciar el algoritmo
mejor_fitness= np.min(I)

# Variable para guardar la historia de cada generacion (para animar)
historia = []

escala = np.abs(rango[0,0] - rango[0,1])

# Seguir con el algoritmo mientra sean menos de 200 generaciones o el fitness sea menor a 0.001
while((mejor_fitness > tol) and (gen < max_gen)):
    
    alpha *= 0.97
    
    # Guardar la posicion de la generacion actual si se va a animar
    if animacion:
        img = np.hstack((X, I.reshape(-1, 1)))
        historia.append(img)
    
    # Actualizar particulas
    for i in range(len(X)):  # Para cada particula
        for j in range(len(X)):  # Comparar con cada particula
            # Checar las intensidades, si la de i es mayor que la de j, se mueve i a j (porque se esta minimizando)
            if I[i] > I[j]:  # Si son iguales, la particula se mueve aleatoriamente
                # Mover la particula i a j
                r = np.sum(np.square(X[i] - X[j]), axis=-1)  # Calcular la distancia entre las particulas
                # Actualizar posicion
                beta = b0 * np.exp(-gamma * r)
                X[i] += beta * (X[j] - X[i]) + alpha * (np.random.rand((d)) - 0.5) * escala
                # clipear la posicion de la particula para que no se salga de los limits
                X[i] = np.clip(X[i], a_min=rango[:, 0], a_max=rango[:, 1])
                # Actualizar la intensidad de la particula i
                I[i] = funcion(X[i].reshape((1, d)))
                
    # Mejor fitness de la poblacion
    mejor_fitness_pop = np.min(I)
    
    # Si el mejor fitness de la poblacion es mejor que el mejor fitness, se actualiza el mejor fitness
    if mejor_fitness_pop < mejor_fitness:
        mejor_fitness = mejor_fitness_pop
        # Obtener la particula con el mejor fitness
        part_opt = X[np.argmin(I)]
    
    # Aumentar la generacion
    gen += 1
    
# Una vez concluido el algoritmo se muestran los resultados
print(f"El óptimo está en {np.around(part_opt, 3)} con un fitness de {mejor_fitness:0.4f}")
print(f"Obtenido en la generación {gen}")

# Graficar la funcion con su minimo menos para colville
if funcion != f.f_colville and not animacion:
    fig, ax = f.graficar(funcion, rango=rango, n_dims=d, mark=list(y_hat[0])+[funcion(y_hat)[0]])
    
elif animacion:
    anim = f.animar_superficie(historia, funcion, rango, d, mark=mark)
    # Mostrar grafica de contorno si la grafica es 3D
    if d == 2:
        anim_c =f.animar_contorno(historia, funcion, rango, mark=mark, levels=levels)

El óptimo está en [0.978 0.957 1.019 1.045] con un fitness de 0.0069
Obtenido en la generación 158
Solo se pueden animar graficas en 3 o menos dimensiones
